In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os

In [2]:
#Root directory of the project
ROOT_DIR = os.path.abspath(".")

# Training file directory
DATASET = os.path.join(ROOT_DIR, 'dataset')
DATASET

'C:\\Users\\Hasib\\Desktop\\works\\lesion-learnt\\dataset'

In [3]:
os.listdir(DATASET)

['ISIC2016']

In [4]:
# ALL IMAGES
IMAGES = os.path.join(DATASET, 'ISIC2016', 'ISBI2016_ISIC_Part3_Training_Data')
# GROUND TRUTH
GT = os.path.join(DATASET, 'ISIC2016', 'ISBI2016_ISIC_Part3_Training_GroundTruth.csv')
len(os.listdir(IMAGES))

900

In [5]:
# Read the metadata
df = pd.read_csv(GT, sep=',', names=["FILE NAME", "CLASS"])
df.head()

,FILE NAME,CLASS
0,ISIC_0000000,benign
1,ISIC_0000001,benign
2,ISIC_0000002,malignant
3,ISIC_0000004,malignant
4,ISIC_0000006,benign


In [6]:
labels = df['CLASS'].values
len(labels)

900

In [7]:
labels[:4]

array(['benign', 'benign', 'malignant', 'malignant'], dtype=object)

In [8]:
labels_num = []

for s in labels:
    if s == "benign":
        labels_num.append(0)
    if s == "malignant":
        labels_num.append(1)
        
len(labels_num)

900

In [9]:
labels_num[:10]

[0, 0, 1, 1, 0, 0, 0, 0, 0, 0]